In [ ]:
!dir

In [1]:
import requests
from bs4 import BeautifulSoup
import os.path
import pandas as pd
import re
import string
BAAsite = 'http://registration.baa.org/'
url_suffix = '/cf/Public/iframe_ResultsSearch.cfm?mode=results'
columns=['bib','name','age','gender','city','state', 'country','ctz',
         't5k','t10k','t15k', 't20k','half','t25k','t30k','t35k','t40k',
         'pace','projt','offt','overcall','gdiv','div']

#only runners are retrived, Handycycle and wheelchair are not included. 
#2014 ma
#gender 0, 1: M, 2: F,

In [2]:
def request_with_year_lastname_gender(year,lastname,gender,start,backoff=1):
    param = {     'StoredProcParamsOn': 'yes',
                 'LastName': lastname,
                 'GenderID': gender,
                 'VarTargetCount': 1000,
                 'records':25,
                 'start': start,
                 "RaceYearLowID": str(year),
                 'next': 'Next+25+Records'}
    url=BAAsite+str(year)+url_suffix
    try:
        requested = requests.post(url, param)
        if requested.status_code != 200:
            raise requests.ConnectionError
    except requests.ConnectionError:
        print ("failed on {}, backing off for {}".format(lastname, backoff))
        if backoff < 128:
            time.sleep(backoff)
            return request_with_year_lastname_gender(year,lastname,gender, backoff*2)
        else:
            print "completely unable to parse {}".format(lastname)
            return None
    return requested

def save_splitdata_tocsv(df_bm,year):
    filename=''.join(['bm_split',str(year),'.csv'])
    if os.path.isfile(filename):
        with open(filename,'a') as f:
            df_bm.to_csv(f,index = False,header=False,encoding='utf-8')
    else:
        df_bm.to_csv(filename,index = False,header=True,encoding='utf-8')
        
def bm_split_list(request_result):
    df_bm=pd.DataFrame(columns=columns)
    soup = BeautifulSoup(request_result.text,"lxml")
    maintable=soup.find_all(name='tr', attrs={'class': 'tr_header'})
    subtable = soup.find_all(name="table", attrs={"class": "table_infogrid"})
    num=len(maintable)
    if num>0:
        for i in range(num):
            (bib,name,age,gender,city,state,country,ctz,_)=[t.text.strip() for t in maintable[i].findAll("td")]
            (t5k, t10k, t15k, t20k,half,t25k,t30k,t35k,t40k,pace,projt,offt,overcall,gdiv,div )= [t.text.strip() for t in subtable[i].findAll("td")]
            df_bm.loc[i]=[bib,name,age,gender,city,state,country,ctz,t5k, t10k, t15k, t20k,half,t25k,t30k,t35k,t40k,pace,projt,offt,overcall,gdiv,div]
    return df_bm, num

In [8]:
string.ascii_lowercase  #python 3? 

'abcdefghijklmnopqrstuvwxyz'

In [9]:
string.lowercase

'abcdefghijklmnopqrstuvwxyz'

In [4]:
def get_list_year_lastname(year, lastname):
    for gender in [1,2]:
        print('lastname:{},gender:{}'.format(lastname,gender)),
        totalnum=0
        for start in list(range(1,1000,25)):
            print('{},'.format(str(start))),
            requested=request_with_year_lastname_gender(year,lastname,gender,start)
            df_bm,num=bm_split_list(requested)
            if num==0: 
                break;
            save_splitdata_tocsv(df_bm,year)
            totalnum+=num
        print('total runner:{}'.format(totalnum))
                

In [ ]:
get_list_year_lastname(2014,'ab')

In [5]:
def get_all_year(year):
    for c1 in string.ascii_lowercase:
        for c2 in string.ascii_lowercase:
            lastname=c1+c2
            get_list_year_lastname(year,lastname)

In [ ]:
!dir

In [6]:
get_all_year(2016)

lastname:aa,gender:1 1, 26, total runner:1
lastname:aa,gender:2 1, 26, total runner:2
lastname:ab,gender:1 1, 26, total runner:19
lastname:ab,gender:2 1, 26, 51, total runner:31
lastname:ac,gender:1 1, 26, total runner:6
lastname:ac,gender:2 1, 26, total runner:12
lastname:ad,gender:1 1, 26, 51, total runner:30
lastname:ad,gender:2 1, 26, total runner:22
lastname:ae,gender:1 1, total runner:0
lastname:ae,gender:2 1, 26, total runner:1
lastname:af,gender:1 1, 26, total runner:3
lastname:af,gender:2 1, 26, total runner:1
lastname:ag,gender:1 1, 26, total runner:12
lastname:ag,gender:2 1, 26, total runner:8
lastname:ah,gender:1 1, 26, total runner:10
lastname:ah,gender:2 1, 26, total runner:5
lastname:ai,gender:1 1, 26, total runner:9
lastname:ai,gender:2 1, 26, total runner:4
lastname:aj,gender:1 1, total runner:0
lastname:aj,gender:2 1, total runner:0
lastname:ak,gender:1 1, 26, total runner:3
lastname:ak,gender:2 1, 26, total runner:5
lastname:al,gender:1 1, 26, 51, 76, 101, 126, total

In [ ]:
!git add BMwithsplits.ipynb

In [ ]:
!git commit -m "add BMwithsplits.ipynb with splits data, version 1"

In [ ]:
!git push

In [7]:
get_all_year(2015)

lastname:aa,gender:1 1, 26, total runner:4
lastname:aa,gender:2 1, total runner:0
lastname:ab,gender:1 1, 26, total runner:25
lastname:ab,gender:2 1, 26, total runner:14
lastname:ac,gender:1 1, 26, total runner:11
lastname:ac,gender:2 1, 26, total runner:11
lastname:ad,gender:1 1, 26, 51, total runner:30
lastname:ad,gender:2 1, 26, 51, total runner:30
lastname:ae,gender:1 1, 26, total runner:2
lastname:ae,gender:2 1, 26, total runner:2
lastname:af,gender:1 1, 26, total runner:3
lastname:af,gender:2 1, 26, total runner:1
lastname:ag,gender:1 1, 26, total runner:15
lastname:ag,gender:2 1, 26, total runner:10
lastname:ah,gender:1 1, 26, total runner:11
lastname:ah,gender:2 1, 26, total runner:6
lastname:ai,gender:1 1, 26, total runner:9
lastname:ai,gender:2 1, 26, total runner:8
lastname:aj,gender:1 1, total runner:0
lastname:aj,gender:2 1, total runner:0
lastname:ak,gender:1 1, 26, total runner:5
lastname:ak,gender:2 1, 26, total runner:2
lastname:al,gender:1 1, 26, 51, 76, 101, total ru

In [10]:
get_all_year(2014)

lastname:aa,gender:1 1, 26, total runner:1
lastname:aa,gender:2 1, 26, total runner:2
lastname:ab,gender:1 1, 26, 51, total runner:32
lastname:ab,gender:2 1, 26, total runner:16
lastname:ac,gender:1 1, 26, total runner:15
lastname:ac,gender:2 1, 26, total runner:14
lastname:ad,gender:1 1, 26, 51, total runner:36
lastname:ad,gender:2 1, 26, 51, total runner:33
lastname:ae,gender:1 1, total runner:0
lastname:ae,gender:2 1, 26, total runner:1
lastname:af,gender:1 1, 26, total runner:3
lastname:af,gender:2 1, 26, total runner:2
lastname:ag,gender:1 1, 26, total runner:15
lastname:ag,gender:2 1, 26, total runner:16
lastname:ah,gender:1 1, 26, total runner:14
lastname:ah,gender:2 1, 26, total runner:9
lastname:ai,gender:1 1, 26, total runner:8
lastname:ai,gender:2 1, 26, total runner:8
lastname:aj,gender:1 1, total runner:0
lastname:aj,gender:2 1, 26, total runner:1
lastname:ak,gender:1 1, 26, total runner:7
lastname:ak,gender:2 1, 26, total runner:12
lastname:al,gender:1 1, 26, 51, 76, 101,

In [11]:
get_all_year(2013)

lastname:aa,gender:1 1, 26, total runner:2
lastname:aa,gender:2 1, 26, total runner:1
lastname:ab,gender:1 1, 26, total runner:19
lastname:ab,gender:2 1, 26, total runner:8
lastname:ac,gender:1 1, 26, total runner:12
lastname:ac,gender:2 1, 26, total runner:8
lastname:ad,gender:1 1, 26, total runner:13
lastname:ad,gender:2 1, 26, total runner:17
lastname:ae,gender:1 1, total runner:0
lastname:ae,gender:2 1, 26, total runner:1
lastname:af,gender:1 1, 26, total runner:2
lastname:af,gender:2 1, 26, total runner:2
lastname:ag,gender:1 1, 26, total runner:14
lastname:ag,gender:2 1, 26, total runner:5
lastname:ah,gender:1 1, 26, total runner:4
lastname:ah,gender:2 1, 26, total runner:3
lastname:ai,gender:1 1, 26, total runner:4
lastname:ai,gender:2 1, 26, total runner:2
lastname:aj,gender:1 1, total runner:0
lastname:aj,gender:2 1, total runner:0
lastname:ak,gender:1 1, 26, total runner:1
lastname:ak,gender:2 1, 26, total runner:4
lastname:al,gender:1 1, 26, 51, 76, total runner:74
lastname: